In [ ]:
import requests
from datetime import datetime
from pymongo import MongoClient

# Setup MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
db = client["stock_prices_db"]

# List of company stock tickers
companies = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Google": "GOOGL",
    "Amazon": "AMZN",
    "Nvidia": "NVDA",
    "Meta": "FB",
    "Tesla": "TSLA"
}

# API key for Polygon.io
api_key = "beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq"

# Date range for querying stock prices
start_date = "2023-06-21"
end_date = "2024-06-21"

for company_name, ticker in companies.items():
    # Construct the API URL
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{start_date}/{end_date}?adjusted=true&sort=asc&apiKey={api_key}"

    # Make the HTTP request to Polygon.io
    response = requests.get(url)
    data = response.json()

    # Check if the response contains data
    if 'results' in data:
        results = data['results']
        formatted_data = []
        for result in results:
            # Convert timestamp to date and time separately
            date_time = datetime.fromtimestamp(result['t'] / 1000)
            date = date_time.strftime('%Y-%m-%d')
            time = date_time.strftime('%H:%M:%S')
            formatted_data.append({
                "Date": date,
                "Time": time,
                "Open": result['o'],
                "High": result['h'],
                "Low": result['l'],
                "Close": result['c'],
                "Volume": result['v']
            })

        # Insert data into MongoDB
        if formatted_data:
            collection = db[company_name]
            # Insert data into the collection
            collection.insert_many(formatted_data)
            print("Data Stored for", company_name)

print("Data fetching and storage complete.")

In [1]:
import yfinance as yf
import pandas as pd

# Define the ticker symbols and the corresponding company names
companies = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Google": "GOOGL",
    "Amazon": "AMZN",
    "Nvidia": "NVDA",
    "Meta": "META",  # Updated ticker symbol from FB to META
    "Tesla": "TSLA"
}

# Define the date range
start_date = "2023-06-21"
end_date = "2024-06-22"

# Create an empty DataFrame to store all the data
all_data = pd.DataFrame()

# Loop through the dictionary and fetch the data for each ticker
for name, ticker in companies.items():
    data = yf.download(ticker, start=start_date, end=end_date, interval="1h")
    data['Ticker'] = ticker  # Add a column for the ticker symbol
    data['Company'] = name   # Add a column for the company name
    # Separate the date and time
    data['Date'] = data.index.date
    data['Time'] = data.index.time
    all_data = pd.concat([all_data, data])

# Drop the original DateTime index
all_data.reset_index(drop=True, inplace=True)

# Sort the data by date and time
all_data.sort_values(by=['Date', 'Time'], inplace=True)

# Save the combined data to a CSV file
all_data.to_csv("stock_data_hourly.csv", index=False)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [2]:
all_data

,Open,High,Low,Close,Adj Close,Volume,Ticker,Company,Date,Time
0,184.899994,185.410004,183.869995,184.119995,184.119995,10433342,AAPL,Apple,2023-06-21,09:30:00
1763,336.369995,337.730011,333.630005,334.320007,334.320007,5706332,MSFT,Microsoft,2023-06-21,09:30:00
3526,122.400002,122.660004,121.079597,121.298897,121.298897,6264229,GOOGL,Google,2023-06-21,09:30:00
5289,125.639999,126.730003,123.870003,124.130096,124.130096,15059357,AMZN,Amazon,2023-06-21,09:30:00
7052,43.382999,43.614998,42.333000,42.635002,42.635002,14787136,NVDA,Nvidia,2023-06-21,09:30:00
...,...,...,...,...,...,...,...,...,...,...
5288,179.960007,180.039993,179.000000,179.630005,179.630005,6082313,GOOGL,Google,2024-06-21,15:30:00
7051,187.050003,189.274994,187.000000,189.080002,189.080002,7529391,AMZN,Amazon,2024-06-21,15:30:00
8814,126.735001,127.070000,125.099998,126.570000,126.570000,40322778,NVDA,Nvidia,2024-06-21,15:30:00
10577,494.730011,495.809998,494.000000,494.779999,494.779999,1439159,META,Meta,2024-06-21,15:30:00
